## Домашнее задание

**На выбор**
* Создать MongoDB, записать туда данные из парсера с предыдущего урока (любое название базы, любое название таблицы). Выполнить команду для демонстрации содержимого коллекции. Прикрепить скриншот.

* Создать базу данных sqlite, загрузить туда данные из парсера с предыдущего урока. Загрузить файл .db

### MongoDB

In [13]:
from pymongo import MongoClient
import json

In [7]:
# Создаем клиента и просматриваем какие базы там лежат
client = MongoClient('mongodb://127.0.0.1:27017')
print(client.list_database_names())

In [10]:
# Создаем новую базу данных quotes
db = client.quotes

In [16]:
# Загружаем данные из файла с цитатами с прошлого урока
with open('Lesson3_v2.json') as f:
    jqoutes = json.load(f)

In [17]:
# Записываем данные в базу данных db = quotes
db.all_quotes.insert_many(jqoutes)

In [18]:
db_q = db.all_quotes.find()
for i in db_q:
    print(i, '\n')

{'_id': ObjectId('626ea0b7af2afb244725caf4'), 'qote_text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'author': 'Albert Einstein', 'author_href': 'https://quotes.toscrape.com/author/Albert-Einstein', 'tags': ['change', 'deep-thoughts', 'thinking', 'world']} 

{'_id': ObjectId('626ea0b7af2afb244725caf5'), 'qote_text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'author': 'J.K. Rowling', 'author_href': 'https://quotes.toscrape.com/author/J-K-Rowling', 'tags': ['abilities', 'choices']} 

{'_id': ObjectId('626ea0b7af2afb244725caf6'), 'qote_text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'author': 'Albert Einstein', 'author_href': 'https://quotes.toscrape.com/author/Albert-Einstein', 'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']} 

{'_id': ObjectId('626ea0b7af2

In [21]:
mir = list(db.all_quotes.find({'tags':'miracle'}))
mir

[{'_id': ObjectId('626ea0b7af2afb244725caf6'),
  'qote_text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'author': 'Albert Einstein',
  'author_href': 'https://quotes.toscrape.com/author/Albert-Einstein',
  'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']}]

### SQLite

**Комментарий**: здесь по хорошему надо создавать нормальную структуру БД, а именно: заводить таблицу на авторов цитат с их idшников, текст цитаты с их idшником и т.п. Ниже приведем упрощенный вариант, когда ко всем цитатам присваивается свой id и так все и складывается в БД. 

In [130]:
import sqlite3

In [131]:
# Считываем данные из json прошлого уровка в jqoutes_for_sql
with open('Lesson3_v2.json') as f:
    jqoutes_for_sql = json.load(f)

# Добавляем id'шник ко всем цитатам
k = 1
for line in jqoutes_for_sql:
    line['id'] = k
    k+=1

In [132]:
connection = sqlite3.connect('sqlite.db')

In [133]:
cursor = connection.cursor()

In [134]:
# Вводим имя таблицы, над которой будем ставить экперименты
table_q_name = 'quotes'

In [135]:
query = f'''
            CREATE TABLE {table_q_name}(
                id INT PRIMARY KEY,
                qote_text TEXT,
                author TEXT,
                author_href TEXT,
                tags TEXT
)'''
cursor.execute(query)
connection.commit()

In [136]:
insert_query = f'''
        INSERT INTO {table_q_name}(
            id,
            qote_text,
            author,
            author_href,
            tags
        )
        VALUES(
            ?,
            ?,
            ?,
            ?,
            ?)         
'''
for line in jqoutes_for_sql:
    if line['tags'] != None:
        tup = (line['id'],line['qote_text'],line['author'],line['author_href'], " ".join(line['tags']))
        cursor.execute(insert_query, tup)
        connection.commit()
    else:
        tup = (line['id'],line['qote_text'],line['author'],line['author_href'], '-')
        cursor.execute(insert_query, tup)
        connection.commit()
print('Data is added to DATABASE successfully')

Data is added to DATABASE successfully


In [138]:
# Проверяем как заполнили таблицу
search_query = f'''SELECT * FROM {table_q_name}'''
cursor.execute(search_query)
rows = cursor.fetchall()
for row in rows:
    print(row, '\n')

(1, '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'Albert Einstein', 'https://quotes.toscrape.com/author/Albert-Einstein', 'change deep-thoughts thinking world') 

(2, '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'J.K. Rowling', 'https://quotes.toscrape.com/author/J-K-Rowling', 'abilities choices') 

(3, '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'Albert Einstein', 'https://quotes.toscrape.com/author/Albert-Einstein', 'inspirational life live miracle miracles') 

(4, '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'Jane Austen', 'https://quotes.toscrape.com/author/Jane-Austen', 'aliteracy books classic humor') 

(5, "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely bor

In [129]:
# Удаляем заполненную таблицу
delete_query = f'''DROP TABLE {table_q_name}'''
cursor.execute(delete_query)
connection.commit()